In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import json

In [2]:
# Load the data from the JSON files
with open('/kaggle/input/hackathon-darpg/problem_statement_1_and_2/no_pii_grievance_v2/no_pii_grievance_v2.json', 'r') as data_file:
    data = json.load(data_file,parse_constant=None)

In [3]:
data[1]['_id']

'GOVUP/E/2023/0000001'

In [4]:
import pandas as pd
import numpy as np

# Initialize an empty list to store rows
x = []

# Open the CSV file for reading
with open('/kaggle/input/mapping-darpg/Mapping.csv', 'r') as file:
    # Iterate over each row in the CSV file
    attr = file.readline().strip().split(';')
#     print(attr)
    for line in file:
        if line.isspace():
            continue
#         print(line)
        fields = line.strip().split(';')
        fields = [value if value != '' else np.nan for value in fields]
        row_data = {attr[i]: value for i, value in enumerate(fields)}
        x.append(row_data)

In [5]:
df = pd.DataFrame(x)
df.head()

,Code,Description,OrgCode
0,1,Telecommunications,DOTEL
1,2,Mobile Related,DOTEL
2,3,Pension Related,DOTEL
3,4,Broadband Related,DOTEL
4,5,Landline Related,DOTEL


In [6]:
df = df.dropna(subset=['OrgCode'])

In [7]:
org_codes = df["OrgCode"].unique()
# invalid = ['16708','6357','6331','16706','6331','5621']

# org_codes = [x for x in org_codes if x not in invalid]
org_codes

array(['DOTEL', 'DPOST', 'DEABD', 'DEAID', 'DOSEL', 'MORTH', 'DHLTH',
       'MEAPD', 'MPANG', 'MOCAV', 'DSEHE', 'MOLBR', 'DOURD', 'MORLY',
       'DOPAT', 'STAFF', 'CBODT', 'CBOEC', 'DORVU', 'MINHA', 'DORLD',
       'DEPOJ', 'DCOYA', 'DOCAF', 'DOWCD', 'MOIAB', 'DDESW', 'FADSS',
       'MINPA', 'DATOM', 'MOSJE', 'ARNPG', 'CAGAO', 'DEAPR', 'DMAFF',
       'MODEF', 'MOSTL', 'DOSAT', 'DHIND', 'MMINE', 'MONRE', 'DBIOT',
       'MCOAL', 'UIDAI', 'MOYAS', 'DSPRT', 'DDRDO', 'MOSPI', 'DSPAC',
       'DPHAM', 'DDPRO', 'MOMAF', 'DOSKD', 'MOPRJ', 'DOIPP', 'MTRBL',
       'DOARE', 'DLGLA', 'DOCOM', 'LGVED', 'DONER', 'MINWR', 'DCLTR',
       'MTXTL', 'DODIV', 'MINIT', 'DOAHD', 'MMSME', 'DODAF', 'MPOWR',
       'DOCPC', 'DOLDR', 'DPUBE', 'DOFPI', 'DHRES', 'DOEAF', 'DOEXP',
       'DFSHR', 'MOCOP', 'DPLNG', 'DTOUR', 'MOEAF', 'DOAAC', 'DOCND',
       'JSOLD', 'DOSIR', 'DOFZR', 'AYUSH', 'DODWS', 'DOFPD', 'MSHPG'],
      dtype=object)

In [8]:
org_labels = {}
for i in range(len(org_codes)):
    org_labels[org_codes[i]] = i+1
    
org_labels['MORLY']

14

In [18]:
orgcode_strings = df.groupby('OrgCode')['Description'].apply(lambda x: ' '.join(x).lower()).to_dict()

In [16]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import spacy

# Load the English model
nlp = spacy.load("en_core_web_sm")

nltk.download('stopwords')

def plural_to_singular(text):
    doc = nlp(text)
    singular_tokens = []
    for token in doc:
        # If the token is a noun and its lemma is different from the original token,
        # consider it a plural form and use its lemma (singular form)
        if token.pos_ == "NOUN" and token.text != token.lemma_:
            singular_tokens.append(token.lemma_)
        else:
            singular_tokens.append(token.text)
    return " ".join(singular_tokens)

delimiters = ['\r','\n','\t','-','?','>>', '/',',','.','!','&',';',':','(',')']

def preprocess(s):
    s = s.lower()
    for x in delimiters:
        s = s.replace(x," ")
    
    s = plural_to_singular(s)
    
    words = word_tokenize(s)
    
    stop_words = set(stopwords.words('english'))
    
    filtered_words = [word for word in words if word.lower() not in stop_words]
    
    tagged_words = pos_tag(filtered_words)
    
    allowed_tags = {'NN', 'NNP','NNS','VB','NNPS'}
    
    filtered_words = [word for word, tag in tagged_words if tag in allowed_tags]
    
    return set(filtered_words)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
for code in org_codes:
    s = orgcode_strings[code]
    w = preprocess(s)
    orgcode_strings[code] = " ".join(w)

orgcode_strings['DOURD']

'technology communication kolkata water development shopping gomtinagar brij comment action person hqtr dharmanagar rani substitution rectification jaipur mobility agency harassment suggestion cd bapudham harrasment gurgaon fixation hour hr sarita housing application ownership interface cipet gr engineering bokaro disaster akhronga marketing face operation ease order complaints block corruption cgewho rikshaw tamilnadu substance misbehaviour facility cost health urban vadodara gurudwara nsnis provision drainage equipments police master sports fssai scale contribution rae regularisation building xix deed crih rajasthan b reservation list aayakar uttrakhand blair establishment handedness programme creche subsidy awareness nbcc nath institutes base family information town woman study grievances cbse amrapali movement chennai museum child issues planning pmay depot ut matter resi+ ltd household minto pm certificate innovation function rera racket occasion cadre milk lsg complition rent bod

In [51]:
data[2]['subject_content_text']

'Labour and Employment >> PF Withdrawal >> Others\r\n\r\nName and Address of Establishment : M/S SKECHERS SOUTH ASIA PVT LTD\r\nUAN No. : Not Provided\r\nPF Code/ PF Account No. : X-X-X-X-X\r\nPPO No. : Not Provided\r\nScheme Certificate Number : Not Provided\r\nPF Office : Regional Office, Thane II\r\n-----------------------\r\nDear sir madam\r\nMy name is RUDRARAM RAJU\r\nMy PF number is X-X-X-X-X\r\nMy company name is M/S SKECHERS SOUTH ASIA PVT LTD\r\n\r\n\r\nSir apply for final settlement but some technical issues I&#39;m not able to apply online so i apply offline final settlement i already send courier pf office kindly check\r\nSir i request you to Epfo department please settled my pf final settlement\r\nPlease find the attachment pdf file'

In [52]:
final_alloted = []
dummy_data = data[0:20]
already_alloted = []

for grievance in dummy_data:
    text = grievance['subject_content_text']
    already_alloted.append(grievance['_id'][0:5])
    
    filtered_words = preprocess(text)
    
#     print(filtered_words)
    
    similarity_score = {}
    
    for code in org_codes:
        similarity_score[code] = 0
    
    for code in org_codes:
        score = 0
        for word1 in filtered_words:
            if word1 in orgcode_strings[code]:
                similarity_score[code]+=1
    final_alloted.append(max(similarity_score,key=lambda x:similarity_score[x]))

print(final_alloted)
print(already_alloted)

['DOURD', 'DOURD', 'MOLBR', 'DOURD', 'DOURD', 'DOURD', 'DOURD', 'DOURD', 'DOURD', 'DOURD', 'DOURD', 'DOURD', 'MOLBR', 'DOURD', 'DOURD', 'DOURD', 'DOURD', 'MOLBR', 'DOURD', 'DOURD']
['MORLY', 'GOVUP', 'MOLBR', 'MOLBR', 'GOVUP', 'GOVUP', 'MODEF', 'DEPOJ', 'MEAPD', 'DORLD', 'DOTEL', 'GOVTG', 'MINHA', 'MOPRJ', 'GOVUP', 'DTOUR', 'GOVTG', 'MOLBR', 'MOLBR', 'GOVTN']


## Word2Vec

In [ ]:
from gensim.models import KeyedVectors
from gensim import models

word2vec_path = '/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin'
w2v_model = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [48]:
dummy_data = data[0:5]
final_alloted = []

for greivance in dummy_data:
    text = greivance['subject_content_text']
    
    filtered_words = preprocess(text)
    print(filtered_words)
    
#     print(filtered_words)
    
    similarity_score = {}
    
    for code in org_codes:
        similarity_score[code] = 0
    
    for code in org_codes:
        score = 0
        length = 0
        for word1 in filtered_words:
            if word1 not in w2v_model:
                continue
            for word2 in orgcode_strings[code]:
                if word2 not in w2v_model:
                    continue
                score += w2v_model.similarity(word1,word2)
                length+=1
        
        similarity_score[code] = score/length
    
    final_alloted.append(max(similarity_score,key=lambda x:similarity_score[x]))
    
final_alloted

{'place', 'side', 'line', 'office', 'yrain', 'condition', 'stone', 'madhyamgram', 'people', 'time', 'passenger', 'zone', 'location', 'please', 'danger', 'railway', 'matter', 'psu', 'cause', 'end', 'look', 'board', 'station', 'pu'}
{'inspection', 'fact', 'p', 'nagar', 'city', 'pressure', 'complaint', 'hpo', 'ji', 'application', 'name', 'xoxpxaxnxnxs', 'post', 'x8x1x6x9x4x', 'shri', 'magistrate', 'virendra', 'kotwali', 'x2x3x', 'tiwari', 'harassment', 'kumar', 'xoxpxnxaxixn', 'cbcid', 'h', 'ansari', 'register', 'phone', 'action', 'sir', 'x0x2x2x', 'evidence', 'person', 'district', 'asp', 'report', 'xuxexixtxnxixg', 'mr', 'postmaster', 'investigate', 'lodge', 'x4x4x0x5x2', 'nath', 'call', 'crime', 'please', 'singh', 'anukriti', 'x', 'permission', 'matter', 'rajeev', 'testimony', 'present', 'agra', 'officer', 'si', 'check', 'bulandshahr', 'fir', 'complainant', 'date', 'x4x4x0x0x3', 'spite', 'offence', 'sharma', 'card', 'x0x2x4x0x0', 'chandra', 'police', 'road', 'prakash'}
{'request', 'esta

['JSOLD', 'JSOLD', 'JSOLD', 'JSOLD', 'JSOLD']